In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as im
from skimage.feature import canny
from skimage.transform import hough_circle, hough_circle_peaks
from time import perf_counter

program_start_time = perf_counter()

# PARAMETER vorher festzulegen
# Das Programm sollte zusammen mit dem ORDNER mit den Bildern
# (path unter image_folder_path abspeichern) in einem Ordner liegen
# low und high sollten für jede Messreihe neu gewählt werden (in exktra Programm),
# Kantenbild anzeigen lassen und so kalibrieren
# potential_radii ist ein array mit potentiellen Radien für die Kugel in Pixeln
# Die Datei Kugel.txt sollte im selben Ordner wie das Programm liegen
input_folder = "originalbilder"
output_file_name = "kugelmitte.txt"
low = 0.2
high = 0.3
r_min = 190
r_max = 210
potential_radii = [206] #np.linspace(r_min,r_max,r_max-r_max+1)  # mögliche Radien

print(os.listdir())
image_files = os.listdir(input_folder)

iteration = 1
with open(output_file_name, "w") as f:
        f.write(f'Name: x: y: r: rmin: rmax: \n')
for file_name in image_files:
    iteration += 1
    iteration_start_time = perf_counter()
    print(f'Reading file {file_name}, {iteration}/{len(image_files)}...')
    image = plt.imread(f'{input_folder}/{file_name}')
    print('Done.')

    # Greyscale
    grey = np.mean(image, -1)

    # Kantenbild
    time_before_edges = perf_counter()
    print(f'Calculating Edges...')
    edges = canny(grey, low_threshold=low, high_threshold=high)
    elapsed_time_edges = perf_counter() - time_before_edges
    print(f'Done after {elapsed_time_edges} seconds.')

    # Hough
    time_before_hough = perf_counter()
    print('Performing Hough Transform...')
    hough = hough_circle(edges, potential_radii)
    # Speichert Eigenschaften des besten Kreis fits
    accums, cx, cy, r = hough_circle_peaks(hough, potential_radii, total_num_peaks=1)
    # Finde index des besten Kreises (r etc. ist ein Array)
    index = np.argwhere(potential_radii == r[0])
    elapsed_time_hough = perf_counter() - time_before_hough
    print(f'Done after {elapsed_time_hough} seconds.')

    x = int(cx[0])
    y = int(cy[0])
    rad = int(r[0])

    # Speichert Hough Bild
    print('Saving Results...')
    im.imsave(f'hough/{file_name}', arr=hough[index][0, 0])

    #Speichert Pruefungs Bild
    print('Saving Circle Mapping...')
    plt.imshow(image[y-2*rad:y+2*rad,x-2*rad:x+2*rad])
    plt.title(f"file_name, x={x}, y={y}, r={rad}")
    plt.gca().add_patch(plt.Circle((2*rad,2*rad), rad, color = 'b', fill = False))
    plt.savefig(f'hough_real/{file_name}') 
    plt.show()

    with open(output_file_name, "a") as f:
        f.write(f'{file_name} {cx[0]} {cy[0]} {r[0]} {r_min} {r_max} \n')

    elapsed_time_iteration = perf_counter() - iteration_start_time
    print(f'Processing of {file_name} done after {elapsed_time_iteration} seconds.')

elapsed_time_program = perf_counter() - program_start_time
print(f'Processed {len(image_files)} images in {elapsed_time_program} seconds.')